# NOCD

In [1]:
import nocd
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import normalize
from tqdm import tqdm
%matplotlib inline

from components.utils import load_edgelist_graph,load_ordered_adjlist_graph
from components.evaluation import modularity, symmetric_matrix_modularity

torch.set_default_tensor_type(torch.cuda.FloatTensor)

## 1、Defining and training the model

Load the dataset
 - `A` (adjacency matrix) is a `scipy.sparse.csr_matrix` of size `[N, N]`
 - `X` (attribute matrix) is a `np.ndarray` of size `[N, D]`

In [2]:
name = 'dpwk'
graph_type = 'merged'

graph_adjlist_path  =  './graphs_' + graph_type + '/' + name + '.adjlist'
graph_edgelist_path =  './graphs_' + graph_type + '/' + name + '.edgelist'
# X directly bio72
X_path = './features/' + name + '.txt'
X = np.loadtxt( X_path, dtype=float)
G_nx = load_ordered_adjlist_graph( graph_adjlist_path )
A = load_edgelist_graph(graph_edgelist_path)
N, D = X.shape
print('x shape:{} * {}'.format(N, D))

100%|██████████| 2852600/2852600 [00:21<00:00, 130904.75it/s]


G nodes 25023,edges 2877623 with self-loops read already
x shape:25023 * 128


### Hyperparameters 

In [3]:
n_clusters = 8
hidden_sizes = [512]                  # hidden sizes of the GNN
weight_decay = 1e-2                   # strength of L2 regularization on GNN weights
dropout = 0.5                         # whether to use dropout
batch_norm = True                     # whether to use batch norm
lr = 1e-3                             # learning rate
max_epochs = 1000                     # number of epochs to train
display_step = 25                     # how often to compute validation loss
balance_loss = True                   # whether to use balanced loss
stochastic_loss = True                # whether to use stochastic or full-batch training
batch_size = 20000                    # batch size (only for stochastic training)

Select & normalize the feature matrix

For some datasets where the features are very informative / correlated with the community structure it's better to use `X` as input (e.g. co-authorship networks w/ keywords as node features). Otherwise, you should try using `A` or `[A, X]` as input.

In [4]:
def gnn_model_train_eval( idx:int):
    
    '''get customized parameters'''
    n_clusters = output_dims[idx]
    
    """x tensor data"""
    x_norm = normalize(X)  # node features
    # x_norm = normalize(A)  # adjacency matrix
    # x_norm = sp.hstack([normalize(X), normalize(A)])  # concatenate A and X
    x_norm = torch.Tensor(x_norm).cuda().to(torch.float32)
    """gnn model"""
    sampler = nocd.sampler.get_edge_sampler(A, batch_size, batch_size, num_workers=5)
    gnn = nocd.nn.GCN(x_norm.shape[1], hidden_sizes, n_clusters, batch_norm=batch_norm, dropout=dropout).cuda()
    # gnn = nocd.nn.ImprovedGCN(x_norm.shape[1], hidden_sizes, n_clusters, dropout=dropout).cuda()
    adj_norm = gnn.normalize_adj(A)
    decoder = nocd.nn.BerpoDecoder(N, A.nnz, balance_loss=balance_loss)
    opt = torch.optim.Adam(gnn.parameters(), lr=lr)
    """train"""
    val_loss = np.inf
    validation_fn = lambda: val_loss
    early_stopping = nocd.train.NoImprovementStopping(validation_fn, patience=10)
    model_saver = nocd.train.ModelSaver(gnn)

    x_display_epochs, val_full_loss_list, x_train_epochs, train_loss_list = [],[],[],[]
    it = iter( sampler )
    # for epoch in tqdm( range( max_epochs+1 ) ):
    for epoch in range( max_epochs+1 ) :
        batch = next(it)
        if epoch % display_step == 0:
            with torch.no_grad():
                gnn.eval()
                # Compute validation loss
                Z = F.relu(gnn(x_norm, adj_norm))
                val_loss = decoder.loss_full(Z, A)
                x_display_epochs.append( epoch )
                val_full_loss_list.append( val_loss )
                # print(f'Epoch {epoch:4d}, loss.full = {val_loss:.4f} ')

                # Check if it's time for early stopping / to save the model
                early_stopping.next_step()
                if early_stopping.should_save():
                    model_saver.save()
                if early_stopping.should_stop():
                    stop_epoch = max_epochs
                    print(f'Breaking due to early stopping at epoch {epoch}')
                    break

        # Training step
        gnn.train()
        opt.zero_grad()
        Z = F.relu(gnn(x_norm, adj_norm))
        ones_idx, zeros_idx = batch
        if stochastic_loss:
            loss = decoder.loss_batch(Z, ones_idx, zeros_idx)
        else:
            loss = decoder.loss_full(Z, A)
        loss += nocd.utils.l2_reg_loss(gnn, scale=weight_decay)
        x_train_epochs.append( epoch )
        train_loss_list.append( loss )
        loss.backward()
        opt.step()
    
    '''evaluate'''
    Z_pred = F.relu(gnn(x_norm, adj_norm))
    # Z_pred = Z.cpu().detach().numpy() > thresh 
    Z_pred = np.argmax(Z_pred.cpu().detach().numpy(), 1)
    model_saver.restore()
    score = symmetric_matrix_modularity(G_nx, Z_pred)
    # print(f'\nFinal symmetric_matrix_modularity = {:.8f}'.format(score))  
    
    '''record'''
    epoch_results.append(x_train_epochs[-1])
    full_loss_results.append(val_full_loss_list[-1])
    eval_results.append(score)
    '''free'''
    torch.cuda.empty_cache()
    '''show parameters and results'''
    print('index:{}'.format(idx),end='\t')
    print('output_dim:{}'.format(output_dims[idx]),end='\t')
    print('epoch_result:{}'.format(epoch_results[idx]),end='\t')
    print('full_loss_results:{}'.format(full_loss_results[idx]),end='\t')
    print('eval_results:{}'.format(eval_results[idx]))


In [5]:
'''customized global parameters'''
output_dims = [18,19,20,21] # ,13,14,15,16,17,18,19,20] # ,16,32,64,128

'''global result records'''
epoch_results = []
full_loss_results = []
eval_results = []

'''work'''
for idx in tqdm(range(len(output_dims))):
    gnn_model_train_eval(idx)

 25%|██▌       | 1/4 [14:15<42:47, 855.93s/it]

index:0	output_dim:18	epoch_result:1000	full_loss_results:0.21708233654499054	eval_results:0.7208499347578388


 50%|█████     | 2/4 [28:19<28:17, 848.71s/it]

index:1	output_dim:19	epoch_result:1000	full_loss_results:0.21665915846824646	eval_results:0.7138786046235402
Breaking due to early stopping at epoch 900


 75%|███████▌  | 3/4 [41:21<13:38, 818.27s/it]

index:2	output_dim:20	epoch_result:899	full_loss_results:0.20927338302135468	eval_results:0.7260651991530943
Breaking due to early stopping at epoch 900


100%|██████████| 4/4 [54:30<00:00, 817.66s/it]

index:3	output_dim:21	epoch_result:899	full_loss_results:0.20924727618694305	eval_results:0.7125074419948373


In [6]:
print(output_dims)
print(eval_results)

[18, 19, 20, 21]
[0.7208499347578388, 0.7138786046235402, 0.7260651991530943, 0.7125074419948373]


## -------------------------------

```
watch loss in train function
print(gnn)
for i in range(len( x_display_epochs )):
    print('epoch {:4d},full_loss:{:6f}'.format( x_display_epochs[i], 
        val_full_loss_list[i] ) )
```

## 2、Analyzing the results

Depending on whether you use balanced loss or not, you should (probably) use different threshold values. From my experience, following are reasonable defaults:
 - for `balance_loss = True`: `thresh = 0.5`
 - for `balance_loss = False`: `thresh = 0.01`
 
You can look at the distribution of the non-zero entries of `Z` to decide on a good value for the threshold. 
I guess it makes sense to select a value that lies in the leftmost "valley" of histogram below. 
You can also look at the unsupervised metrics in the next section of this notebook to make an informed choice.

Note that all of these are just speculations based on the behavior that I observed for a handful of datasets, YMMV.

### Visualize the adjacency matrix sorted by the communities

```
# 损失函数随迭代次数变化
plt.plot(x_train_epochs, train_loss_list, label='train_loss_list')
plt.title('training loss value towards epochs')
plt.show()

plt.title('validating value towards epochs')
plt.plot(x_display_epochs, val_full_loss_list, label='val_full_loss_list')
plt.show()

plt.hist(Z[Z > 0].cpu().detach().numpy(), 100);

plt.figure(figsize=[12, 12])
o = np.argsort(Z_pred)
nocd.utils.plot_sparse_clustered_adjacency(A, n_clusters, Z_pred, o, markersize=0.05)
```

```
# Sizes of detected communities
print(Z_pred.sum(0))
```

### Quantify quality of the communities based on unsupervised metrics.

Metrics:
* **Coverage**: what percentage of the edges is explained by at least one community? (i.e. if $(u, v)$ is an edge, both nodes share at least one community) Higher is better.
$$\textrm{Coverage}(C_1, ..., C_K) = \frac{1}{|E|}\sum_{u, v \in E} \mathbb{1}[z_u^T z_v > 0]$$


* **Density**: average density of the detected communities (weighted by community size). Higher is better.

$$\rho(C) = \frac{\text{# existing edges in $C$}}{\text{# of possible edges in $C$}}$$

$$\textrm{AvgDensity}(C_1, ..., C_K) = \frac{1}{\sum_i |C_i|}\sum_i \rho(C_i) \cdot |C_i|$$


* **Conductance**: average conductance of the detected communities (weighted by community size). Lower is better.

$$\textrm{outside}(C) = \sum_{u \in C, v \notin C} A_{uv}$$

$$\textrm{inside}(C) = \sum_{u \in C, v \in C, v \ne u} A_{uv}$$

$$\textrm{Conductance}(C) = \frac{\textrm{outside}(C)}{\textrm{inside}(C) + \textrm{outside}(C)}$$

$$\textrm{AvgConductance}(C_1, ..., C_K) = \frac{1}{\sum_i |C_i|}\sum_i \textrm{Conductance}(C_i) \cdot |C_i|$$


* **Clustering coefficient**: average clustering coefficient of the detected communities (weighted by community size). Higher is better.

$$\textrm{ClustCoef}(C) = \frac{\text{# existing triangles in $C$}}{\text{# of possible triangles in $C$}}$$

$$\textrm{AvgClustCoef}(C_1, ..., C_K) = \frac{1}{\sum_i |C_i|}\sum_i \textrm{ClustCoef}(C_i) \cdot |C_i|$$

Clustering coefficient & density of the entire graph

```
density_baseline = A.nnz / (N**2 - N)
num_triangles = (A @ A @ A).diagonal().sum() / 6
num_possible_triangles = (N - 2) * (N - 1) * N / 6
clust_coef_baseline = num_triangles / num_possible_triangles
print(f'Background (over the entire graph):\n'
      f' - density    = {density_baseline:.3e}\n'
      f' - clust_coef = {clust_coef_baseline:.3e}')
```

```
### Ground truth communities & Predicted communities
metrics = nocd.metrics.evaluate_unsupervised(Z_gt, A)
print(f"Ground truth communities:\n"
      f" - coverage    = {metrics['coverage']:.4f}\n"
      f" - conductance = {metrics['conductance']:.4f}\n"
      f" - density     = {metrics['density']:.3e}\n"
      f" - clust_coef  = {metrics['clustering_coef']:.3e}")
metrics = nocd.metrics.evaluate_unsupervised(Z_pred, A)
print(f"Predicted communities:\n"
      f" - coverage    = {metrics['coverage']:.4f}\n"
      f" - conductance = {metrics['conductance']:.4f}\n"
      f" - density     = {metrics['density']:.3e}\n"
      f" - clust_coef  = {metrics['clustering_coef']:.3e}")
The detected partition has lower conductance / higher density / higher clustering coefficient than the GT communities.
```